In [ ]:
import math
import statistics

import numpy as np

%run game.ipynb

## Score

In [ ]:
@cache
def calculate_tile_score(tile: Tile) -> int:
    """
    Obtiene el score correspondiente a la baldosa dada.
    :param tile: Baldosa.
    :return: Score de la baldosa.
    """
    if not tile: return 1
    return tile * (math.log2(tile) - 1)


def calculate_score(board: Board) -> int:
    """
    Obtiene el score del tablero dado.
    :param board: Tablero.
    :return: Score del tablero.
    """
    return sum(calculate_tile_score(tile) for tile in board.flat)

## Celdas vacías

In [ ]:
def count_empty_cells(board: Board) -> int:
    """
    Obtiene el número de celdas vacías del tablero dado.
    :param board: Tablero.
    :return: Celdas vacías del tablero.
    """
    return len(get_empty_cells(board))

## Similitud

In [ ]:
@cache
def get_neighbor_cells(cell: Cell) -> filter:
    """
    Obtiene las celdas vecinas pertenecientes al tablero de la celda dada.
    :param cell: Celda.
    :return: Celdas vecinas.
    """
    vectors = [
        (-1, -1),
        (-1, 0),
        (-1, 1),
        (0, -1),
        (0, 1),
        (1, -1),
        (1, 0),
        (1, 1)
    ]

    return filter(
        lambda c: in_board(c),
        map(
            lambda v: sum_vectors(v, cell),
            vectors
        )
    )


def calculate_similarity(board: Board) -> int:
    """
    Obtiene la similitud del tablero dado.
    :param board: Tablero
    :return: Similitud del tablero.
    """
    similarity = 0

    for cell, tile in np.ndenumerate(board):
        if not tile: continue

        neighbor_cells = get_neighbor_cells(cell)
        neighbor_tiles = map(lambda c: board[cell], neighbor_cells)

        # Filtrar celdas vacías
        neighbor_tiles = filter(lambda t: t, neighbor_tiles)
        differences = tuple(map(lambda t: abs(t - tile), neighbor_tiles))

        # Si ha habido celdas ocupadas.
        if len(differences):
            similarity += statistics.fmean(differences)

    return similarity

## Monotonía

In [ ]:
def calculate_monotony(board: Board) -> int:
    """
    Obtiene la monotonía del tablero dado.
    :param board: Tablero.
    :return: Monotonía del tablero.
    """
    monotony_matrix = np.array([
        [7, 6, 5, 4],
        [6, 5, 4, 3],
        [5, 4, 3, 2],
        [4, 3, 2, 1]
    ], dtype=np.uint8
    )

    return np.multiply(board, monotony_matrix).sum()

## Baldosa máxima

In [ ]:
def get_max_tile(board: Board) -> Tile:
    """
    Obtiene le valor de la baldosa máxima.
    :param board: Tablero.
    :return: Baldosa máxima.
    """
    return np.max(board)

# Heurísticas propias

Estas son heurísticas que se me han ocurrido.

## Número de fusiones posibles

In [ ]:
def count_possible_merges(board: Board) -> int:
    """
    Obtiene el número de baldosas iguales que están una al lado de la otra en el tablero dado.
    No las tiene en cuenta si hay espacios entre medias.
    :param board: Tablero.
    :return: Número combinaciones posibles.
    """
    count = 0

    for cell, tile in np.ndenumerate(board):
        for direction in (Direction.DOWN, Direction.RIGHT):
            next_cell = sum_vectors(cell, direction.vector)
            if in_board(next_cell) and tile == board[next_cell]:
                count += 1

    return count

## Monotonía en forma de serpiente

In [ ]:
def calculate_snake_monotony(board: Board) -> int:
    """
    Obtiene la monotonía del tablero dado con una monotonía en forma de serpiente.
    :param board: Tablero.
    :return: Monotonía del tablero.
    """
    monotony_matrix = np.array([
        [255, 128, 64, 32],
        [2, 4, 8, 16],
        [1, 0, 0, 0],
        [0, 0, 0, 0]
    ], dtype=np.uint8
    )

    return np.multiply(board, monotony_matrix).sum()

## Número de movimientos posibles

In [ ]:
def count_possible_moves(board: Board) -> int:
    return len(get_possible_moves(board))